In [1]:
import pandas as pd

In [56]:
dataset = pd.read_csv('ready_dataset.csv')

In [61]:
dataset.drop(columns = 'Unnamed: 0')

,tipologia,genere,zona,bagno,cucina,salotto,balcone,range euro
0,doppia,M/F,Bolognina,1,1,0,0,225-250
1,doppia,F,Galvani,1,1,0,0,300-325
2,singola,M/F,Bolognina,2,1,0,1,350-375
3,doppia,M,Mazzini,1,1,1,1,275-300
4,singola,M,San Donato,1,1,0,1,350-375
...,...,...,...,...,...,...,...,...
195,singola,M/F,San Donato,1,1,1,0,450-475
196,doppia,F,San Donato,1,1,0,1,200-225
197,doppia,F,Saffi,1,1,1,0,225-250
198,doppia,F,Mazzini,1,1,1,0,250-275


In [74]:
array = dataset.values
X = array[:,0:8]
X

array([[0, 'doppia', 'M/F', ..., 1, 0, 0],
       [1, 'doppia', 'F', ..., 1, 0, 0],
       [2, 'singola', 'M/F', ..., 1, 0, 1],
       ...,
       [197, 'doppia', 'F', ..., 1, 1, 0],
       [198, 'doppia', 'F', ..., 1, 1, 0],
       [199, 'singola', 'F', ..., 1, 0, 0]], dtype=object)

In [98]:
Y = array[:,8]
Y

array(['225-250', '300-325', '350-375', '275-300', '350-375', '400-425',
       '250-275', '450-475', '325-350', '400-425', '275-300', '275-300',
       '325-350', '425-450', '350-375', '350-375', '550-575', '300-325',
       '250-275', '450-475', '250-275', '225-250', '300-325', '450-475',
       '225-250', '400-425', '350-375', '225-250', '450-475', '300-325',
       '250-275', '400-425', '375-400', '400-425', '325-350', '200-225',
       '325-350', '350-375', '250-275', '250-275', '300-325', '200-225',
       '325-350', '250-275', '500-525', '225-250', '350-375', '200-225',
       '450-475', '325-350', '400-425', '225-250', '425-450', '225-250',
       '350-375', '400-425', '425-450', '300-325', '275-300', '275-300',
       '350-375', '500-525', '450-475', '225-250', '325-350', '300-325',
       '350-375', '400-425', '350-375', '275-300', '350-375', '200-225',
       '250-275', '400-425', '400-425', '300-325', '275-300', '225-250',
       '350-375', '225-250', '400-425', '300-325', 

In [105]:
tipologia = ['singola', 'doppia']
genere = ['F', 'M', 'M/F']
zona = ['Barca', 'Bolognina', 'Borgo Panigale', 'Colli', 'Corticella', 'Costa Saragozza', 'Galvani', 'Irnerio', 'Lame', 'Malpighi', 'Marconi', 'Mazzini', 'Murri', 'S.Ruffillo', 'S.Viola', 'S.Vitale', 'Saffi', 'San Donato']

In [106]:
tipologia

['singola', 'doppia']

In [107]:
genere

['F', 'M', 'M/F']

In [110]:
zona

['Barca',
 'Bolognina',
 'Borgo Panigale',
 'Colli',
 'Corticella',
 'Costa Saragozza',
 'Galvani',
 'Irnerio',
 'Lame',
 'Malpighi',
 'Marconi',
 'Mazzini',
 'Murri',
 'S.Ruffillo',
 'S.Viola',
 'S.Vitale',
 'Saffi',
 'San Donato']

In [112]:
>>> from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

In [109]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score